In [1]:
import pandas as pd
import numpy as np
import flow

df = flow.get_energy_production_data()
df.head()

,Data_Status,StateCode,MSN,1960,1961,1962,1963,1964,1965,1966,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,2018F,AK,BDFDB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,21.0,27.0,29.0,15.0
1,2018F,AK,BDPRP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,5.0,5.0,3.0
2,2018F,AK,BFFDB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,21.0,27.0,29.0,15.0
3,2018F,AK,BFPRP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,5.0,5.0,3.0
4,2018F,AK,CLPRB,11263.0,11497.0,13588.0,13307.0,11622.0,13931.0,14461.0,...,29016.0,33556.0,33524.0,31332.0,24917.0,22944.0,17747.0,13942.0,14365.0,13752.0
